In [60]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
from scipy.stats import f_oneway
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
#freesurfer_dir='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

# Load in Beta values

In [24]:
subj='subj01'
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', 'lh.betas_nsdsynthetic.hdf5')
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
task_average=True

roi_vals = [prep.convert_between_roi_num_and_vareas("V1"), [1,2,3,4,5]]
roi_path = [os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label', f'lh.prf-{roi_file}.mgz') for roi_file in ["visualrois", "eccrois"]]
mask, roi_dict = prep.load_common_mask_and_rois(roi_path, roi_vals)

In [77]:
betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                          stim_info_path=stim_info,
                          task_keys=task_keys, task_average=task_average,
                          x_axis='voxel', y_axis='stim_idx', long_format=True)


In [81]:
betas_df['sub'] = subj
avg_betas_df = betas_df.groupby(['sub','freq_lvl','voxel']).mean().reset_index()

In [83]:
betas_df.query('voxel == 0 & freq_lvl == 0')

,stim_idx,image_idx,names,w_r,w_a,phase,class_idx,freq_lvl,voxel,task,betas,sub
0,0,104,pinwheel,0.0,6.0,0.000000,0,0,0,avg,1.321667,subj01
227021,1,105,pinwheel,0.0,6.0,1.570796,0,0,0,avg,2.753333,subj01
454042,2,106,pinwheel,0.0,6.0,3.141593,0,0,0,avg,1.098333,subj01
681063,3,107,pinwheel,0.0,6.0,4.712389,0,0,0,avg,1.440000,subj01
5448504,24,128,forward spiral,4.0,4.0,0.000000,6,0,0,avg,0.658333,subj01
5675525,25,129,forward spiral,4.0,4.0,1.570796,6,0,0,avg,0.933333,subj01
5902546,26,130,forward spiral,4.0,4.0,3.141593,6,0,0,avg,0.831667,subj01
6129567,27,131,forward spiral,4.0,4.0,4.712389,6,0,0,avg,1.463333,subj01
10897008,48,152,annulus,6.0,0.0,0.000000,12,0,0,avg,0.796667,subj01
11124029,49,153,annulus,6.0,0.0,1.570796,12,0,0,avg,0.338333,subj01


In [84]:
n = 0
one_voxel_betas_df = betas_df.query('voxel == @n')
f_oneway(one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 0].betas,
         one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 1].betas,
         one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 2].betas,
         one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 3].betas,
         one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 4].betas,
         one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == 5].betas)

F_onewayResult(statistic=4.6204450381415825, pvalue=0.0007461423675273558)

In [87]:
one_voxel_betas_df.freq_lvl.unique().tolist()

[0, 1, 2, 3, 4, 5]

In [90]:
test = [one_voxel_betas_df[one_voxel_betas_df['freq_lvl'] == k].betas for k in range(6)]
F, p = f_oneway(*test)
p

0.0007461423675273558